In [1]:
# Import Dependencies
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
# Import haul cycle info


haul_prfl = "Resources\MSN_lom_Profiles.xlsx"



haul_profile_df = pd.read_excel(haul_prfl)

haul_profile_df

,PB,Bench,Dump,Dump_Z,Tons,V Lift (+),V Lift (-),H Dist,lift_wt,drop_wt,dist_w
0,MH,1770,2610 Goat Trail Dump (In-Pit),2610,321.00,840,0,11126,269640.0,0.0,3571446.00
1,MH,1770,CR1,3150,295.21,1380,0,15842,407389.8,0.0,4676716.82
2,MH,1770,CR3,3240,261.79,1470,0,23398,384831.3,0.0,6125362.42
3,MH,1810,2610 Goat Trail Dump (In-Pit),2610,704.00,800,0,11380,563200.0,0.0,8011520.00
4,MH,1810,CR1,3150,648.19,1340,0,16096,868574.6,0.0,10433266.24
...,...,...,...,...,...,...,...,...,...,...,...
659,MH,3450,3490 Minearl Hill Dump,3490,1401.00,40,0,4554,56040.0,0.0,6380154.00
660,MH,3490,3450 Minearl Hill Dump,3450,1499.00,0,40,6723,0.0,59960.0,10077777.00
661,MH,3530,3450 Minearl Hill Dump,3450,1110.00,0,80,6492,0.0,88800.0,7206120.00
662,MH,3570,3450 Minearl Hill Dump,3450,329.00,0,120,6467,0.0,39480.0,2127643.00


In [3]:
# Clean profile sheet Drop NUll Values

haul_profile_df.dropna(inplace=True)

In [4]:
# Groupby PB, Bench, Dump

profiles_grouped = haul_profile_df.groupby(by=['PB','Bench','Dump','Dump_Z']).sum()

profiles_grouped.reset_index(inplace=True)

In [5]:
profiles_grouped

,PB,Bench,Dump,Dump_Z,Tons,V Lift (+),V Lift (-),H Dist,lift_wt,drop_wt,dist_w
0,MH,1770,2610 Goat Trail Dump (In-Pit),2610,321.00,840,0,11126,269640.0,0.0,3571446.00
1,MH,1770,CR1,3150,295.21,1380,0,15842,407389.8,0.0,4676716.82
2,MH,1770,CR3,3240,261.79,1470,0,23398,384831.3,0.0,6125362.42
3,MH,1810,2610 Goat Trail Dump (In-Pit),2610,704.00,800,0,11380,563200.0,0.0,8011520.00
4,MH,1810,CR1,3150,648.19,1340,0,16096,868574.6,0.0,10433266.24
...,...,...,...,...,...,...,...,...,...,...,...
476,VL,3090,CR3,3240,39.95,150,0,7306,5992.5,0.0,291874.70
477,VL,3130,3330 East Dump,3330,1235.00,240,40,17415,296400.0,49400.0,21507525.00
478,VL,3130,3650 East Pima Dump,3650,200.00,560,40,13694,112000.0,8000.0,2738800.00
479,VL,3170,3330 East Dump,3330,718.00,240,80,16717,172320.0,57440.0,12002806.00


In [6]:
# Calc grouped average lift, drop, and dist

profiles_grouped['V Lift (+)'] = profiles_grouped['lift_wt']/profiles_grouped['Tons']
profiles_grouped['V Lift (-)'] = profiles_grouped['drop_wt']/profiles_grouped['Tons']
profiles_grouped['H Dist'] = profiles_grouped['dist_w']/profiles_grouped['Tons']
    
profiles_grouped['int'] =  profiles_grouped['V Lift (+)'] + profiles_grouped['Bench']

profiles_export = profiles_grouped.drop(columns=['lift_wt', 'drop_wt', 'dist_w','Tons', 'V Lift (+)', 'V Lift (-)'], axis=1)

In [7]:
profiles_export

,PB,Bench,Dump,Dump_Z,H Dist,int
0,MH,1770,2610 Goat Trail Dump (In-Pit),2610,11126.0,2610.0
1,MH,1770,CR1,3150,15842.0,3150.0
2,MH,1770,CR3,3240,23398.0,3240.0
3,MH,1810,2610 Goat Trail Dump (In-Pit),2610,11380.0,2610.0
4,MH,1810,CR1,3150,16096.0,3150.0
...,...,...,...,...,...,...
476,VL,3090,CR3,3240,7306.0,3240.0
477,VL,3130,3330 East Dump,3330,17415.0,3370.0
478,VL,3130,3650 East Pima Dump,3650,13694.0,3690.0
479,VL,3170,3330 East Dump,3330,16717.0,3410.0


In [8]:
profiles_export = profiles_export.reindex(columns=['PB', 'Bench', 'Dump', 'Dump_Z','int','H Dist'])

profiles_export

,PB,Bench,Dump,Dump_Z,int,H Dist
0,MH,1770,2610 Goat Trail Dump (In-Pit),2610,2610.0,11126.0
1,MH,1770,CR1,3150,3150.0,15842.0
2,MH,1770,CR3,3240,3240.0,23398.0
3,MH,1810,2610 Goat Trail Dump (In-Pit),2610,2610.0,11380.0
4,MH,1810,CR1,3150,3150.0,16096.0
...,...,...,...,...,...,...
476,VL,3090,CR3,3240,3240.0,7306.0
477,VL,3130,3330 East Dump,3330,3370.0,17415.0
478,VL,3130,3650 East Pima Dump,3650,3690.0,13694.0
479,VL,3170,3330 East Dump,3330,3410.0,16717.0


In [9]:
file_path = "Resources\profiles_expt.csv"

profiles_export.to_csv(file_path)

In [11]:
cone_profiles = profiles_export

cone_profiles

,PB,Bench,Dump,Dump_Z,int,H Dist
0,MH,1770,2610 Goat Trail Dump (In-Pit),2610,2610.0,11126.0
1,MH,1770,CR1,3150,2610.0,15842.0
2,MH,1770,CR3,3240,3240.0,23398.0
3,MH,1810,2610 Goat Trail Dump (In-Pit),2610,2610.0,11380.0
4,MH,1810,CR1,3150,2610.0,16096.0
...,...,...,...,...,...,...
476,VL,3090,CR3,3240,3240.0,7306.0
477,VL,3130,3330 East Dump,3330,3370.0,17415.0
478,VL,3130,3650 East Pima Dump,3650,3690.0,13694.0
479,VL,3170,3330 East Dump,3330,3410.0,16717.0


In [18]:
owl = []

for x in cone_profiles['Dump']:
    if x == "CR1" or x=="CR3":
        y = 1
    else:
        y= 0
    owl.append(y)

cone_profiles["OWL"] = owl
cone_profiles

,PB,Bench,Dump,Dump_Z,int,H Dist,OWL
0,MH,1770,2610 Goat Trail Dump (In-Pit),2610,2610.0,11126.0,0
1,MH,1770,CR1,3150,2610.0,15842.0,1
2,MH,1770,CR3,3240,3240.0,23398.0,1
3,MH,1810,2610 Goat Trail Dump (In-Pit),2610,2610.0,11380.0,0
4,MH,1810,CR1,3150,2610.0,16096.0,1
...,...,...,...,...,...,...,...
476,VL,3090,CR3,3240,3240.0,7306.0,1
477,VL,3130,3330 East Dump,3330,3370.0,17415.0,0
478,VL,3130,3650 East Pima Dump,3650,3690.0,13694.0,0
479,VL,3170,3330 East Dump,3330,3410.0,16717.0,0


In [ ]:
cone_export = cone_profiles.groupby(by=['PB','OWL'])